# 3. Get the Context to See Who Else Got the Email

Notebook to build all of the different stix objects associated with finding who else got the suspicious email

We take v3.0 of the [Incident Core Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/incident-core/Incident%20Extension%20Suite.adoc) specification, with particular focus on the Sighting Extensions for Evidence, and combine it with the standard Stix approach. There are nine stages:

1. First setup the global parameters and retrieve the context memory
2. Retrieve the user-account, email address and identity for each of the users who recieved the email
2. Add the SCO objects to the `ObservedData` object
4. Create the `Sighting` object with the `SightingContext` extension, using their identity's as the location
5. Add the Sighting to the original Event
5. Create the next Task and its Sequence objects
6. Connect the Sequence object to the previous sequence object
6. Create the Incident, with the [Incident Core Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/incident-core/Incident%20Extension%20Suite.adoc), and add all of the objects to it
7. Save the context memory


Each time a block makes an object, and saves it as a json, this notebook will parse the object into an actual Stix object, so it can be bundled and printy printed (This step also verifies the objects are created correctly, and is cool).



## A. Load Imports

### A.1 StixORM Imports


In [ ]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, Incident, URL, EmailMessage, ObservedData
)
from stixorm.module.definitions.os_threat import (
    EventCoreExt, Event, SocialMediaContact, ContactNumber, IncidentCoreExt, TaskCoreExt,
    Task, SightingEvidence, Sequence, SequenceExt, AnecdoteExt, Anecdote,
    SightingAnecdote, SightingAlert, SightingContext, SightingExclusion,
    SightingEnrichment, SightingHunt, SightingFramework, SightingExternal
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

### A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [ ]:
import sys
sys.path.append('../')
import os

### A.3 Relative Import of Blocks

In [ ]:
import json
from Block_Families.StixORM.SCO.EmailMessage.make_email_message import main as make_email_msg
from Block_Families.StixORM.SRO.Relationship.make_sro import main as make_sro
from Utilities.local_make_general import invoke_save_options_context_block, invoke_save_incident_context_block, invoke_get_from_options_block, invoke_get_from_incident_block, invoke_move_unattached_to_other_block
from Utilities.local_make_sro import invoke_sro_block, invoke_sighting_block
from Utilities.local_make_sdo import (
    invoke_make_observed_data_block, invoke_make_indicator_block, invoke_make_event_block, invoke_make_sequence_block,
    invoke_make_task_block, invoke_make_incident_block, invoke_make_impact_block
)
from Utilities.local_make_sco import (
    invoke_make_email_addr_block, invoke_make_url_block, invoke_make_e_msg_block, invoke_make_anecdote_block
)
from conv import conv

context_base = "../Orchestration/Context_Mem/"
path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_derived.json"
context_data = {
    "usr2": "sbilly@example.com",
    "usr3": "wwhilly@example.com",
    "usr4": "strange@mycompany.com",
    "usr5": "dumbo@mycompany.com",
}

TR_Context_Memory_Path = "./Context_Mem/Type_Refinery_Context.json"

## B. Collect the SCO's Based on Searching the Exchange Server

### B.1 Load the Email Addresses

In [ ]:
# Create the Queries to get the User Details

context_type = {
    "context_type": "users"
}
usr1_email_query = {
    "type" : "email-addr",
    "property": {
        "path": ["value"],
        "source_value": "naive@example.com",
        "comparator": "EQ"
    }
}
usr1_email = invoke_get_from_options_block(usr1_email_query, context_type, source_value=None, source_id=None)
usr2_email_query = {
    "type" : "email-addr",
    "property": {
        "path": ["value"],
        "source_value": context_data["usr2"],
        "comparator": "EQ"
    }
}
usr2_email = invoke_get_from_options_block(usr2_email_query, context_type, source_value=None, source_id=None)
usr3_email_query = {
    "type" : "email-addr",
    "property": {
        "path": ["value"],
        "source_value": context_data["usr3"],
        "comparator": "EQ"
    }
}

usr3_email = invoke_get_from_options_block(usr3_email_query, context_type, source_value=None, source_id=None)
usr4_email_query = {
    "type" : "email-addr",
    "property": {
        "path": ["value"],
        "source_value": context_data["usr4"],
        "comparator": "EQ"
    }
}
usr4_email = invoke_get_from_options_block(usr4_email_query, context_type, source_value=None, source_id=None)
usr5_email_query = {
    "type" : "email-addr",
    "property": {
        "path": ["value"],
        "source_value": context_data["usr5"],
        "comparator": "EQ"
    }
}
usr5_email = invoke_get_from_options_block(usr5_email_query, context_type, source_value=None, source_id=None)

#
# Save everything but the existing indicator down itno unattached
context_type = {
    "context_type": "unattached"
}
sro2_results_path = results_base + "step3/observation-context_context.json"
sro2_context_results_path = results_base + "step3/observation-context_context.json"
result = invoke_save_incident_context_block(sro2_results_path, sro2_context_results_path, context_type)
print(f"SRO2 result->{result}")



### B.2 Retrieve the User Account and Identity for each email address

for each user who got the phishing email

In [ ]:
# 1. For user 2, get the user account and identity

usr2_usr_acct_query = {
    "type" : "user-account",
    "property": {
        "path": ["belongs_to_ref "],
        "source_value": usr2_email["id"],
        "comparator": "EQ"
    }
}
usr2_acct = invoke_get_from_options_block(usr2_usr_acct_query, context_type, source_value=None, source_id=None)
usr3_usr_acct_query = {
    "type" : "user-account",
    "property": {
        "path": ["belongs_to_ref"],
        "source_value": usr3_email["id"],
        "comparator": "EQ"
    }
}
usr3_acct = invoke_get_from_options_block(usr3_usr_acct_query, context_type, source_value=None, source_id=None)
usr4_usr_acct_query = {
    "type" : "user-account",
    "property": {
        "path": ["belongs_to_ref"],
        "source_value": usr4_email["id"],
        "comparator": "EQ"
    }
}
usr4_acct = invoke_get_from_options_block(usr4_usr_acct_query, context_type, source_value=None, source_id=None)
usr5_usr_acct_query = {
    "type" : "user-account",
    "property": {
        "path": ["belongs_to_ref"],
        "source_value": usr5_email["id"],
        "comparator": "EQ"
    }
}
usr5_acct = invoke_get_from_options_block(usr3_usr_acct_query, context_type, source_value=None, source_id=None)
usr2_identity_query = {
    "type" : "identity",
    "property": {
        "path": ["extensions", "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498", "email_addresses", "email_address_ref"],
        "source_value": usr2_email["id"],
        "comparator": "EQ"
    }
}
usr2_identity = invoke_get_from_options_block(usr2_identity_query, context_type, source_value=None, source_id=None)
usr3_identity_query = {
    "type" : "identity",
    "property": {
        "path": ["extensions", "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498", "email_addresses", "email_address_ref"],
        "source_value": usr3_email["id"],
        "comparator": "EQ"
    }
}
usr3_identity = invoke_get_from_options_block(usr3_identity_query, context_type, source_value=None, source_id=None)
usr4_identity_query = {
    "type" : "identity",
    "property": {
        "path": ["extensions", "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498", "email_addresses", "email_address_ref"],
        "source_value": usr4_email["id"],
        "comparator": "EQ"
    }
}
usr4_identity = invoke_get_from_options_block(usr4_identity_query, context_type, source_value=None, source_id=None)
usr5_identity_query = {
    "type" : "identity",
    "property": {
        "path": ["extensions", "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498", "email_addresses", "email_address_ref"],
        "source_value": usr5_email["id"],
        "comparator": "EQ"
    }
}
usr5_identity = invoke_get_from_options_block(usr5_identity_query, context_type, source_value=None, source_id=None)
context_type = {
    "context_type": "systems"
}
exchange_identity_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": "Microsoft Exchange",
        "comparator": "EQ"
    }
}
exchange_identity = invoke_get_from_options_block(exchange_identity_query, context_type, source_value=None, source_id=None)
context_type = {
    "context_type": "systems"
}
exchange_identity_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": "Microsoft Exchange",
        "comparator": "EQ"
    }
}
exchange_identity = invoke_get_from_options_block(exchange_identity_query, context_type, source_value=None, source_id=None)
context_type = {
    "context_type": "other"
}
indicator_query = {
    "type" : "indicator",
    "property": {
        "path": ["name"],
        "source_value": "Potential Phishing Email",
        "comparator": "EQ"
    }
}
indicator = invoke_get_from_incident_block(indicator_query, context_type, source_value=None, source_id=None)
#
# Save everything but the existing indicator down itno unattached
context_type = {
    "context_type": "unattached"
}
context_records = [
    "Usr1__email",
    "Usr1__ident",
    "Usr1__usr_acct",
    "Usr2__email",
    "Usr2__ident",
    "Usr2__usr_acct",
    "Usr3__email",
    "Usr3__ident",
    "Usr3__usr_acct",
    "Usr4__email",
    "Usr4__ident",
    "Usr4__usr_acct",
    "Usr5__email",
    "Usr5__ident",
    "Usr5__usr_acct",
    "Systems_Laptop1__ident",
    "Systems_Laptop2__ident",
    "Systems_Laptop3__ident",
    "Systems_Laptop4__ident",
    "Systems_Laptop5__ident",
    "Systems_Exchange__ident"
]
for rec in context_records:
    results_path = results_base + "step0/" + rec + ".json"
    save_path = results_base + "step3/" + rec + "_context.json"
    result = invoke_save_incident_context_block(results_path, save_path, context_type)
    print(f"{rec} result->{result}")

results_path = results_base + "step1/indicator_context" + ".json"
save_path = results_base + "step3/indicator_context" + "_context.json"
result = invoke_save_incident_context_block(results_path, save_path, context_type)
print(f"{rec} result->{result}")

## B Make the SRO's to Connect all Who Receieved the Email

In [ ]:

# B.1 Setup SRO objectsrelationship_type = "derived-from"
relationship_type = "duplicate-of"
sro_data_path = "SRO/Relationship/sro_derived.json"
sro2_results_path = "step3/SRO2_derived.json"
sro2_context_results_path = "step3/SRO2_derived_context.json"
sro2 = invoke_sro_block(sro_data_path, sro2_results_path, usr1_email, usr2_email, relationship_type)
result = invoke_save_incident_context_block(sro2_results_path, sro2_context_results_path, context_type)
print(f"SRO2 result->{result}")
sro3_results_path = "step3/SRO3_derived.json"
sro3_context_results_path = "step3/SRO3_derived_context.json"
sro3 = invoke_sro_block(sro_data_path, sro3_results_path, usr1_email, usr3_email, relationship_type)
result = invoke_save_incident_context_block(sro3_results_path, sro3_context_results_path, context_type)
print(f"SRO3 result->{result}")
sro4_results_path = "step3/SRO4_derived,json"
sro4_context_results_path = "step3/SRO4_derived_context.json"
sro4 = invoke_sro_block(sro_data_path, sro4_results_path, usr1_email, usr4_email, relationship_type)
result = invoke_save_incident_context_block(sro4_results_path, sro4_context_results_path, context_type)
print(f"SRO4 result->{result}")
sro5_results_path = "step3/SRO5_derived.json"
sro5_context_results_path = "step3/SRO5_derived_context.json"
sro5 = invoke_sro_block(sro_data_path, sro5_results_path, usr1_email, usr5_email, relationship_type)
result = invoke_save_incident_context_block(sro5_results_path, sro5_context_results_path, context_type)
print(f"SRO5 result->{result}")

## C. Collect the Elements of the Observation into an Observed-Data Object

Tbis then represents the potential phishing email as a group of elements

In [ ]:
# 1. Setup observed-data
obs_refs3 = [usr2_acct.id, usr2_email.id, usr2_identity.id, usr3_email.id, usr3_acct.id, usr3_identity.id,              usr4_email.id, usr4_acct.id, usr4_identity.id,usr5_email.id, usr5_acct.id, usr5_identity.id,
            sro2.id, sro3.id, sro4.id, sro5.id]
# 2. Setup path to form and results
obs_path ="SDO/Observed_Data/observation-context.json"
results_path ="step3/observation-context.json"
# 3. Invoke the Make Observed Data Block
obs_3 = invoke_make_observed_data_block(obs_path, results_path, observation=obs_refs3)
# 4. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
obs_results_context_path = results_base + "step3/observation-context_context.json"
result = invoke_save_incident_context_block(results_base + results_path, obs_results_context_path, context_type)
print(f" result->{result}")

## D. Create the Contextual Sighting

Tbis then represents the Anecdote on Impact from the reporting user

### D.1 Create the Sighting
Connecting the:
- observed-data object, containing the extra identitiies, email addresses and user accounts that got the email
- the exchange server as the location
- the original indicator and pattern

In [ ]:

#
# D. Setup the Sighting Object
sighting_data_path ="SRO/Sighting/sighting_context.json"
results_path ="step3/sighting_context.json"
# 2. Setup the SDO sighted, the Observed-Data that was observed with generated objects,
#                        then the identity object from the context storage (note the slight difference in indexing
sighted = indicator
observation_list = [obs_3]
where_list = [exchange_identity]
# 2. Invoke the Make Observed Data Block
sight3 = invoke_sighting_block(sighting_data_path, results_path, observed=observation_list, sighted=sighted, where=where_list)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
sighting_results_context_path = results_base + "step1/sighting_context_context.json"
result = invoke_save_incident_context_block(results_base + results_path, sighting_results_context_path, context_type)
print(f" result->{result}")

### D.2 Promote The Sighting and its Components from Unattached to Other

Everything that uis connected in the Sighting should now be promoted, so move Unattached to the Other list

In [ ]:
total_list = [usr2_acct, usr2_email, usr2_identity, usr3_email, usr3_acct, usr3_identity,              usr4_email, usr4_acct, usr4_identity,usr5_email, usr5_acct, usr5_identity,
            sro2, sro3, sro4, sro5]
total_list.append(obs_3)
total_list.append(indicator)
total_list.append(sight3)
total_list.append(exchange_identity)

obs_context_move_path = results_base + "step3/context/context_move.json"
obs_context_move_results = results_base + "step3/context/context_move_results.json"
result = invoke_move_unattached_to_other_block(obs_context_move_path, obs_context_move_results, total_list)

## E. Create the Impact Objects

Tbis Impact represents the effect reported by the user in the Anecdote

In [ ]:
# 1. Setup path to form and results
impact_path ="SDO/Impact/context_impact.json"
results_path ="step3/impact_context.json"
# 2. Setup the number of assets impacted
numbers = {"computers-mobile": 5}
impacted_refs =
# 2. Invoke the Make Observed Data Block
impact_1 = invoke_make_impact_block(impact_path, results_path, impacted_entity_counts=numbers, impacted_refs=impacted_refs, superseded_by_ref=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "impact"
}
impact_results_obj_path = results_base + results_path
impact_results_context_path = results_base + "/step2/impact_anecdote_context.json"
result = invoke_save_incident_context_block(impact_results_obj_path, impact_results_context_path, context_type)
print(f" result->{result}")

## F. Create the Next Task Object

Next step is to investigate the Exchange server to see who else got the email

### F.1 Create the Task Object

In [ ]:
# 1. Setup path to form and results
task_data_path ="SDO/Task/task_anecdote.json"
results_path ="step2/task_anecdote.json"
# 2. Invoke the Make Observed Data Block
task_2 = invoke_make_task_block(task_data_path, results_path, changed_objects=None)
# 3. Add the record to the in-session bundles and lists
bundle_list = bundle_list + task_2
task_objs.append(task_2[0])
#
# Step 3.A.2 New Task to check te Exclusion Lists in Step 4
#
task3 = Task(
    task_types=["investigation"], outcome="pending", name="Check Exclusion Lists",
    description="Check OS-Threat Exclusion List to see if email address is a known phisher",
    owner=me.id, extensions={task_ext_id:task_ext}
)
tseq1_3 = Sequence(
    step_type="single_step", sequenced_object=task3.id,
    sequence_type="task", extensions=seq_ext_dict
)

### F.2 Create the Sequence Object

In [ ]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_alert.json"
results_path ="step1/sequence_task_single.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "single_step"
sequence_type = "task"
sequenced_object = task_2[0]["id"]
# 2. Invoke the Make Observed Data Block
seq_T_2 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=sequenced_object, on_completion=None, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
bundle_list = bundle_list + seq_T_2
sequence_objs.append(seq_T_2[0])

### F.3 Connect the Sequence to the Previous one

In [ ]:
# Find the Previous Sequence and link it to the above Sequence
last_task = task_objs[-1]
for rec in sequence_objs:
    if rec["sequence_type"] == "task" and rec["sequenced_object"] == last_task["id"]:
        rec["on_completion"] = seq_T_2[0]["id"]

## G. Finally, Append all of the objects to the Incident Object

Assign all of the objets to the Incident Object

In [ ]:
# 1. Setup path to form and results
inc_path ="SDO/Incident/phishing_incident.json"
results_path ="step1/incident_alert.json"
# 2. Setup the Sequence Object for the Event
#
sequence_start_refs = [x["id"] for x in sequence_start_objs]
sequence_refs = [x["id"] for x in sequence_objs]
task_refs = [x["id"] for x in task_objs]
event_refs = [x["id"] for x in event_objs]
impact_refs = [x["id"] for x in impact_objs]
other_object_refs = [x["id"] for x in other_object_objs]
# 3. Update the actual Incident Object
incident_obj["sequence_start_refs"] = sequence_start_refs
incident_obj["sequence_refs"] = sequence_refs
incident_obj["task_refs"] = task_refs
incident_obj["event_refs"] = event_refs
incident_obj["impact_refs"] = impact_refs
incident_obj["other_object_refs"] = other_object_refs

## H. Write out the Contex Memory for the Incident

Export out the Context Memory for the Incident

In [ ]:
# Save the Tpe Refinery Context Memory File
invoke_context_save_block(Type_Refinery_Context)